In [1]:
from pathlib import Path
import pandas as pd
from smount_predictors import SeamountTransformer, SeamountHelp, SeamountCVSplitter
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import plotly.express as px
import numpy as np

In [2]:
pipe = Pipeline([
    ('trans', SeamountTransformer()),
    ('predictor', SVC(kernel='linear', class_weight='balanced', C=1.0))  # previous grid searches have found optimal C of 1
])

In [ ]:
points = SeamountHelp.readKMLbounds(Path('data/seamount_training_zone.kml'))
data = SeamountHelp.readAndFilterGRD(Path('data/test_files') / 'training_data_new.nc')
X = data.to_dataframe().reset_index()[['lat', 'lon' , 'z']]

OSError: [Errno -51] NetCDF: Unknown file format: '/Users/martinhawks/Code/Git_repos/seamount_picking/code/data/training_data_new.nc'

In [ ]:
y = data.to_dataframe().reset_index()['Labels']
pipe.fit(X, y)
print(f'train score: {pipe.score(X, y)}')

train score: 0.9525945216049383


In [ ]:
import pickle
from sklearn.cluster import DBSCAN
from smount_predictors.src.SeamountHelp import PipelinePredictor

full_pipeline = PipelinePredictor(pipe, DBSCAN(eps=0.00029088820866630336, min_samples=4, metric='haversine'))
pickle.dump(full_pipeline, open('out/cluster_tuned_model.pkl', 'wb'))